# QUESTIONS AND ANSWER GENERATOR FROM PDF


In [ ]:
%%writefile requirements.txt

langchain
langchain-community
llama-parse
fastembed
chromadb
python-dotenv
langchain-groq
chainlit
fastembed
unstructured[md]

Overwriting requirements.txt


In [ ]:
! pip install -r requirements.txt

In [ ]:
##### LLAMAPARSE #####
from llama_parse import LlamaParse

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
#
from groq import Groq
from langchain_groq import ChatGroq
#
import joblib
import os
import nest_asyncio  # noqa: E402
nest_asyncio.apply()

In [ ]:


from dotenv import load_dotenv

# Load variables from .env file
load_dotenv('.env')


True

In [ ]:
llamaparse_api_key = os.getenv('LLAMA_API_KEY')
groq_api_key = os.getenv("GROQ_API_KEY")

In [ ]:
!mkdir data

In [ ]:

def load_or_parse_data():
    data_file = "./data/parsed_data.pkl"

    if os.path.exists(data_file):
        # Load the parsed data from the file
        parsed_data = joblib.load(data_file)
    else:
        # Perform the parsing step and store the result in llama_parse_documents
        parsingInstruction = """ It contains tables.
        Try to be precise while generating the questions and answers. """
        parser = LlamaParse(api_key=llamaparse_api_key,
                            result_type="markdown",
                            parsing_instruction=parsingInstruction,
                            max_timeout=10000,)
        llama_parse_documents = parser.load_data("./data/Decision_tree.pdf")


        # Save the parsed data to a file
        print("Saving the parse results in .pkl format ..........")
        joblib.dump(llama_parse_documents, data_file)

        # Set the parsed data to the variable
        parsed_data = llama_parse_documents


    print(parsed_data)
    return parsed_data



In [ ]:
pd=load_or_parse_data()

Started parsing the file under job_id 5e261940-313f-48ef-856c-f30e2302e456
Saving the parse results in .pkl format ..........
[Document(id_='83d70dfe-de2f-4020-b277-76a0f61ce44e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='# Decision Trees Lecture\n\n# Lecture 7: Decision Trees\n\n|Contents|\n|---|\n|1 Learning Goals|3|\n|2 Examples of Decision Trees|3|\n|3 Definition and Classifying an Example|7|\n|3.1 What is a decision tree?|7|\n|3.2 Classifying an example using a decision tree|7|\n|4 The Decision Tree Learning Algorithm|8|\n|4.1 Issues in learning a decision tree|8|\n|4.2 Grow a full tree given an order of testing features|8|\n|4.3 When do we stop?|11|\n|4.4 Base case 2: no features left|11|\n|4.5 Base case 3: no examples left|13|\n|4.6 Pseudo-code for the decision tree learner algorithm|15|\n|5 Determine the Order of Testing Features|16|\n|5.1 Which feature should we test at each step?|16|\n|5.2 Identifying 

In [ ]:
type(pd)

list

In [ ]:
pd

In [ ]:
# Create vector database
def create_vector_database():
    """
    Creates a vector database using document loaders and embeddings.

    This function loads urls,
    splits the loaded documents into chunks, transforms them into embeddings using OllamaEmbeddings,
    and finally persists the embeddings into a Chroma vector database.

    """
    # Call the function to either load or parse the data
    llama_parse_documents = load_or_parse_data()
    #print(llama_parse_documents[0].text[:300])

    with open('data/output.md', 'a') as f:  # Open the file in append mode ('a')
        for doc in llama_parse_documents:
            f.write(doc.text + '\n')

    markdown_path = "/content/data/output.md"
    loader = UnstructuredMarkdownLoader(markdown_path)

   #loader = DirectoryLoader('data/', glob="**/*.md", show_progress=True)
    documents = loader.load()
    # Split loaded documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    docs = text_splitter.split_documents(documents)

    #len(docs)
    print(f"length of documents loaded: {len(documents)}")
    print(f"total number of document chunks generated :{len(docs)}")
    #docs[0]

    # Initialize Embeddings
    embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

    # Create and persist a Chroma vector database from the chunked documents
    vs = Chroma.from_documents(
        documents=docs,
        embedding=embed_model,
        persist_directory="chroma_db_llamaparse1",  # Local mode with in-memory storage only
        collection_name="rag"
    )


    print('Vector DB created successfully !')
    return vs,embed_model

In [ ]:
vs,embed_model = create_vector_database()

[Document(id_='83d70dfe-de2f-4020-b277-76a0f61ce44e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='# Decision Trees Lecture\n\n# Lecture 7: Decision Trees\n\n|Contents|\n|---|\n|1 Learning Goals|3|\n|2 Examples of Decision Trees|3|\n|3 Definition and Classifying an Example|7|\n|3.1 What is a decision tree?|7|\n|3.2 Classifying an example using a decision tree|7|\n|4 The Decision Tree Learning Algorithm|8|\n|4.1 Issues in learning a decision tree|8|\n|4.2 Grow a full tree given an order of testing features|8|\n|4.3 When do we stop?|11|\n|4.4 Base case 2: no features left|11|\n|4.5 Base case 3: no examples left|13|\n|4.6 Pseudo-code for the decision tree learner algorithm|15|\n|5 Determine the Order of Testing Features|16|\n|5.1 Which feature should we test at each step?|16|\n|5.2 Identifying the most important feature|16|\n|5.3 Entropy of a distribution over two outcomes|17|\n|5.4 Expected information gain of testin

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


length of documents loaded: 1
total number of document chunks generated :36


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

Vector DB created successfully !


In [ ]:
chat_model = ChatGroq(temperature=0.0,
                      model_name="mixtral-8x7b-32768",
                      api_key=groq_api_key)

In [ ]:
vectorstore = Chroma(embedding_function=embed_model,
                      persist_directory="chroma_db_llamaparse1",
                      collection_name="rag")

retriever=vectorstore.as_retriever(search_kwargs={'k': 3})

In [ ]:
custom_prompt_template = """Use the following pieces of information to answer questions of the user.

Context: {context}
Question: {question}

Only return the helpful content below and nothing else.
Helpful answer:
"""

In [ ]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt
#
prompt = set_custom_prompt()


########################### RESPONSE ###########################
PromptTemplate(input_variables=['context', 'question'], template=custom_prompt_template)

PromptTemplate(input_variables=['context', 'question'], template='Use the following pieces of information to answer questions of the user.\n\nContext: {context}\nQuestion: {question}\n\nOnly return the helpful content below and nothing else.\nHelpful answer:\n')

In [ ]:
qa = RetrievalQA.from_chain_type(llm=chat_model,
                               chain_type="stuff",
                               retriever=retriever,
                               return_source_documents=True,
                               chain_type_kwargs={"prompt": prompt})

In [ ]:
response = qa.invoke({
    "query": "Generate 20 technical interview questions and answers suitable for a candidate with 0 year of experience "
             "in the field, based on the provided content. Include a mix of basic, intermediate, tricky, and logical "
             "questions. Follow a coherent order in the question formation. Provide the source documents "
})


In [ ]:
temp_str=response['result']

In [ ]:
lines = temp_str.split("\n")

In [ ]:
lines

['Sure, here are 20 technical interview questions and answers based on the provided content:',
 '',
 '1. What is a decision tree and how does it correspond to a program?',
 'Answer: A decision tree is a flowchart-like structure that is used to classify or predict a target variable based on input features. It corresponds to a program with a big nested if-then-else structure.',
 '2. What are the advantages of using a decision tree?',
 'Answer: Decision trees are easy to understand, interpret, and visualize. They can handle both numerical and categorical data, and they can be used for both classification and regression tasks.',
 '3. What are the disadvantages of using a decision tree?',
 'Answer: Decision trees can be prone to overfitting, especially if they are too deep. They can also be sensitive to small variations in the data, leading to different trees being generated for different training sets.',
 '4. What is decision tree learning algorithm?',
 'Answer: Decision tree learning algo

In [ ]:
response2 = qa.invoke({
    "query": "Generate 20 technical interview questions and answers suitable for a candidate with 1 year of experience "
             "in the field, based on the provided content. Include a mix of basic, intermediate, tricky, and logical "
             "questions. Follow a coherent order in the question formation. "
})

In [ ]:
lines2=response2['result'].split("\n")
lines2

['Sure, here are 20 technical interview questions and answers based on the provided content:',
 '',
 '1. What is a decision tree and how is it used in machine learning?',
 'Answer: A decision tree is a flowchart-like structure that is used to classify or predict examples in machine learning. Each internal node represents a feature, each branch represents a decision rule, and each leaf node represents an outcome.',
 '2. What is the decision tree learning algorithm?',
 'Answer: The decision tree learning algorithm is a method for building a decision tree from a dataset. It involves selecting the best feature to split the data at each node, recursively, until a stopping criterion is met.',
 '3. What are some issues in learning a decision tree?',
 'Answer: Some issues in learning a decision tree include overfitting, underfitting, noisy data, missing data, and irrelevant features.',
 '4. What is the order of testing features in the Jeeves data set?',
 'Answer: The order of testing features 

In [ ]:
response3 = qa.invoke({
    "query": "Generate 20 technical interview questions and answers suitable for a candidate with 5 year of experience "
             "in the field, based on the provided content. Include a mix of basic, intermediate, tricky, and logical "
             "questions. Follow a coherent order in the question formation. Do not add extra information by yourself"
})

In [ ]:
lines3=response3['result'].split("\n")
lines3

['Decision Tree Learning Questions:',
 '',
 '1. Q: What is a decision tree in machine learning?',
 '   A: A decision tree is a flowchart-like structure used for making decisions and predicting outcomes. Each internal node represents a feature or attribute, each branch represents a decision rule, and each leaf node represents an outcome.',
 '',
 '2. Q: What is the purpose of decision tree learning?',
 '   A: The goal of decision tree learning is to create a model that can predict the value of a target variable by learning simple decision rules inferred from the data features.',
 '',
 '3. Q: What are the key challenges in learning a decision tree?',
 '   A: Deciding the order of testing input features, handling missing values, and avoiding overfitting are some of the key challenges in learning a decision tree.',
 '',
 '4. Q: How does a decision tree correspond to a program?',
 '   A: A decision tree corresponds to a program with a big nested if-then-else structure, where each internal no

In [ ]:
lines4=response4['result'].split("\n")
lines4

['When learning a decision tree, choosing a feature to test at each step by maximizing the expected information gain does not guarantee an optimal decision tree. This is because the approach is greedy and does not consider the future consequences of the current decision. It is possible that a suboptimal split at one step may lead to a better overall tree, but this is not guaranteed. Additionally, binary splits only further limit the possibilities of finding an optimal tree.']